In [1]:
import sys
from pathlib import Path

parent_dir = Path().resolve().parent

# Перевірити, чи вже доданий шлях, і додати його, якщо ні
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

parent_dir = str(parent_dir).replace("\\", "/") + "/"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("eda_ukrainian_price").getOrCreate()

# Ukraine Price Dataset

In [ ]:
from data.schema.ukraine_price_dataset import INITIAL_SCHEMA, UKRAINE_PRICE_INITIAL_COLUMNS

csv_file_path = parent_dir + "data/raw/ukraine_price_dataset.csv"

df = (
    spark.read.format("csv")
    .option("header", "true")
    .schema(INITIAL_SCHEMA)
    .load(csv_file_path)
)

In [4]:
df.show(5)

+--------------------+--------------------+---------------------+--------------------+-------------+--------+----------------------+--------------+-------------+---------------------------+----------------+----------------------+----------------------+
|            Показник|     Базисний період|Територіальний розріз|Тип товарів і послуг|Періодичність|  Період|Значення спостереження|Одиниця виміру|Масштабування|Кількість десяткових знаків|Конфіденційність|Примітки часового ряду|Примітки спостереження|
+--------------------+--------------------+---------------------+--------------------+-------------+--------+----------------------+--------------+-------------+---------------------------+----------------+----------------------+----------------------+
|Базовий індекс сп...|До відповідного м...|              Україна|   Не застосовується|      Місячна|2008-M01|                  NULL|      Відсоток|      Одиниці|                       Один|            NULL|  1. З 2010 року./F...|  1. Відомос

## Extract Metadata Into Columns

In [5]:
df.select(UKRAINE_PRICE_INITIAL_COLUMNS.TIME_SERIES_NOTES).distinct().show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import regexp_extract, when, col, lit


class UKRAINE_PRICE_METADATA_COLUMN:
    FROM_YEAR = "from_year"
    CORE_INFLATION_DESCRIPTION = "core_inflation_description"
    CORE_INFLATION_CALCULATION = "core_inflation_calculation"
    DATA_EXCLUSION_2014 = "data_exclusion_2014"
    DATA_EXCLUSION_2015_2021 = "data_exclusion_2015_2021"
    DATA_EXCLUSION_2022_2024 = "data_exclusion_2022_2024"
    CLASSIFICATION = "classification"
    DATA_REVIEW = "data_review"
    REGION_CODES = "region_codes"


patterns = {
    "core_inflation_description": "Базовий індекс споживчих цін",
    "core_inflation_calculation": "Базовий індекс споживчих цін до відповідного періоду попереднього року",

    "data_exclusion_2014": "Дані за 2014 рік наведено без урахування тимчасово окупованої території Автономної Республіки Крим та м.Севастополя",

    "data_exclusion_2015_2021": "Дані за 2015-2021 роки наведено без урахування тимчасово окупованої території Автономної Республіки Крим, м.Севастополя та частини тимчасово окупованих територій у Донецькій та Луганській областях",
    "data_exclusion_2022_2024": "Дані за 2022-2024 роки наведено без урахування тимчасово окупованих російською федерацією територій та частини територій, на яких ведуться (велися) бойові дії",
    "classification": "Класифікація індивідуального споживання за цілями (КІСЦ)",
    "data_review": "Перегляд даних, розрахованих і оприлюднених раніше, не здійснюється",
    "region_codes": "Коди регіонів відповідно до Кодифікатора адміністративно-територіальних одиниць та територій територіальних громад",
}



# Витягнення початкового року



df = df.withColumn(
    "from_year",
    regexp_extract(

        col(UKRAINE_PRICE_INITIAL_COLUMNS.TIME_SERIES_NOTES), r"1\.\s*З\s+(\d{4})", 1
    ).cast("int"),
)



# Створення булевих колонок на основі наявності певних патернів



for col_name, pattern in patterns.items():

    df = df.withColumn(
        col_name,
        when(

            col(UKRAINE_PRICE_INITIAL_COLUMNS.TIME_SERIES_NOTES).contains(pattern), lit(True)
        ).otherwise(lit(False)),
    )



# Опціонально: Видалення оригінальної колонки, якщо вона більше не потрібна



df = df.drop(UKRAINE_PRICE_INITIAL_COLUMNS.TIME_SERIES_NOTES)

In [7]:
df.select(
    "from_year",
    "core_inflation_description",
    "core_inflation_calculation",
    "data_exclusion_2014",
    "data_exclusion_2015_2021",
    "data_exclusion_2022_2024",
    "classification",
    "data_review",
    "region_codes",
).tail(20)

[Row(from_year=2001, core_inflation_description=False, core_inflation_calculation=False, data_exclusion_2014=False, data_exclusion_2015_2021=False, data_exclusion_2022_2024=False, classification=True, data_review=True, region_codes=True),
 Row(from_year=2001, core_inflation_description=False, core_inflation_calculation=False, data_exclusion_2014=False, data_exclusion_2015_2021=False, data_exclusion_2022_2024=False, classification=True, data_review=True, region_codes=True),
 Row(from_year=2001, core_inflation_description=False, core_inflation_calculation=False, data_exclusion_2014=False, data_exclusion_2015_2021=False, data_exclusion_2022_2024=False, classification=True, data_review=True, region_codes=True),
 Row(from_year=2001, core_inflation_description=False, core_inflation_calculation=False, data_exclusion_2014=False, data_exclusion_2015_2021=False, data_exclusion_2022_2024=False, classification=True, data_review=True, region_codes=True),
 Row(from_year=2001, core_inflation_descript

## Analyze "Примітки спостереження" column

In [8]:
df.select(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES).distinct().show(truncate=False)

+--------------------------------------------------+
|Примітки спостереження                            |
+--------------------------------------------------+
|1. Відомості відсутні./Not available.             |
|1. Явищ не було./Not observed.                    |
|1. Визначено експертним шляхом./Expert`s estimate.|
|NULL                                              |
+--------------------------------------------------+



In [ ]:
df = df.where(
    (
        col(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES)
        != "1. Відомості відсутні./Not available."
    )
    | col(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES).isNull()
)

In [ ]:
df = df.where(
    (col(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES) != "1. Явищ не було./Not observed.")
    | col(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES).isNull()
)

In [11]:
df.count()

194780

In [ ]:
df = df.drop(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_NOTES)

## Remove CONFIDENTIALITY column

In [13]:
df.select(UKRAINE_PRICE_INITIAL_COLUMNS.CONFIDENTIALITY).distinct().show()

+----------------+
|Конфіденційність|
+----------------+
|            NULL|
+----------------+



In [ ]:
df = df.drop(UKRAINE_PRICE_INITIAL_COLUMNS.CONFIDENTIALITY)

## Review And Split Cleaned Data

In [15]:
df.show(10)

+--------------------+--------------------+---------------------+--------------------+-------------+--------+----------------------+--------------+-------------+---------------------------+---------+--------------------------+--------------------------+-------------------+------------------------+------------------------+--------------+-----------+------------+
|            Показник|     Базисний період|Територіальний розріз|Тип товарів і послуг|Періодичність|  Період|Значення спостереження|Одиниця виміру|Масштабування|Кількість десяткових знаків|from_year|core_inflation_description|core_inflation_calculation|data_exclusion_2014|data_exclusion_2015_2021|data_exclusion_2022_2024|classification|data_review|region_codes|
+--------------------+--------------------+---------------------+--------------------+-------------+--------+----------------------+--------------+-------------+---------------------------+---------+--------------------------+--------------------------+-------------------

In [16]:
df.count()

194780

In [17]:
# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------+---------------+---------------------+--------------------+-------------+------+----------------------+--------------+-------------+---------------------------+---------+--------------------------+--------------------------+-------------------+------------------------+------------------------+--------------+-----------+------------+
|Показник|Базисний період|Територіальний розріз|Тип товарів і послуг|Періодичність|Період|Значення спостереження|Одиниця виміру|Масштабування|Кількість десяткових знаків|from_year|core_inflation_description|core_inflation_calculation|data_exclusion_2014|data_exclusion_2015_2021|data_exclusion_2022_2024|classification|data_review|region_codes|
+--------+---------------+---------------------+--------------------+-------------+------+----------------------+--------------+-------------+---------------------------+---------+--------------------------+--------------------------+-------------------+------------------------+------------------------+------

## Change Columns DataTypes

In [ ]:
df = df.select(
    col(UKRAINE_PRICE_INITIAL_COLUMNS.INDICATOR).alias("indicator"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.BASE_PERIOD).alias("base_period"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.TERRITORIAL_BREAKDOWN).alias("territorial_breakdown"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.TYPE_OF_GOODS_AND_SERVICES).alias("type_of_goods_and_services"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.FREQUENCY).alias("frequency"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.PERIOD).alias("period"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.OBSERVATION_VALUE).cast("double").alias("observation_value"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.UNIT_OF_MEASUREMENT).alias("unit_of_measurement"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.SCALING).alias("scaling"),
    col(UKRAINE_PRICE_INITIAL_COLUMNS.NUMBER_OF_DECIMAL_PLACES).alias("number_of_decimal_places"),
    col(UKRAINE_PRICE_METADATA_COLUMN.FROM_YEAR).cast("integer"),
    col(UKRAINE_PRICE_METADATA_COLUMN.CORE_INFLATION_DESCRIPTION),
    col(UKRAINE_PRICE_METADATA_COLUMN.CORE_INFLATION_CALCULATION),
    col(UKRAINE_PRICE_METADATA_COLUMN.DATA_EXCLUSION_2014),
    col(UKRAINE_PRICE_METADATA_COLUMN.DATA_EXCLUSION_2015_2021),
    col(UKRAINE_PRICE_METADATA_COLUMN.DATA_EXCLUSION_2022_2024),
    col(UKRAINE_PRICE_METADATA_COLUMN.CLASSIFICATION),
    col(UKRAINE_PRICE_METADATA_COLUMN.DATA_REVIEW),
    col(UKRAINE_PRICE_METADATA_COLUMN.REGION_CODES),
)

In [19]:
df.printSchema()

root
 |-- indicator: string (nullable = true)
 |-- base_period: string (nullable = true)
 |-- territorial_breakdown: string (nullable = true)
 |-- type_of_goods_and_services: string (nullable = true)
 |-- frequency: string (nullable = true)
 |-- period: string (nullable = true)
 |-- observation_value: double (nullable = true)
 |-- unit_of_measurement: string (nullable = true)
 |-- scaling: string (nullable = true)
 |-- number_of_decimal_places: string (nullable = true)
 |-- from_year: integer (nullable = true)
 |-- core_inflation_description: boolean (nullable = false)
 |-- core_inflation_calculation: boolean (nullable = false)
 |-- data_exclusion_2014: boolean (nullable = false)
 |-- data_exclusion_2015_2021: boolean (nullable = false)
 |-- data_exclusion_2022_2024: boolean (nullable = false)
 |-- classification: boolean (nullable = false)
 |-- data_review: boolean (nullable = false)
 |-- region_codes: boolean (nullable = false)



## Save Data

In [21]:
df.write.option("header", True).option("delimiter", ",").csv(
    parent_dir + "data/processed/ukraine_price_dataset.csv"
)

In [23]:
unique_values = df.select('indicator').distinct()
unique_values.show(truncate=False)

+-----------------------------------------+
|indicator                                |
+-----------------------------------------+
|Індекс споживчих цін                     |
|Базовий індекс споживчих цін             |
|Середні споживчі ціни на товари (послуги)|
+-----------------------------------------+



Індекс споживчих цін (ІСЦ): Характеризує зміни у часі загального рівня цін на товари та послуги, які купує населення для невиробничого споживання.

Базовий індекс споживчих цін (БІСЦ): Вимірює інфляцію, виключаючи волатильні категорії, такі як харчові продукти та енергоносії.

Середні споживчі ціни на товари (послуги): Відображає середній рівень цін у різних секторах економіки, таких як житло та транспорт.

In [24]:
df_cpi = df.filter(df['indicator'] == "Індекс споживчих цін")
df_bcpi = df.filter(
    df['indicator'] == "Базовий індекс споживчих цін"
)
df_acpg = df.filter(
    df['indicator'] == "Середні споживчі ціни на товари (послуги)"
)

In [25]:
df_cpi.write.option("header", True).option("delimiter", ",").csv(
    parent_dir + "data/processed/splited_data/index_consuming_prices.csv"
)

In [26]:
df_bcpi.write.option("header", True).option("delimiter", ",").csv(
    parent_dir + "data/processed/splited_data/base_index_consuming_prices.csv"
)

In [27]:
df_acpg.write.option("header", True).option("delimiter", ",").csv(
    parent_dir + "data/processed/splited_data/average_consuming_goods_prices.csv"
)

# USD to UAH Dataset

In [3]:
from data.schema import STOCK_PRICE_SCHEMA,STOCK_PRICE_COLUMNS

csv_file_path = parent_dir + "data/raw/usd_uah_currency.csv"

df = (
    spark.read.format("csv")
    .option("header", "true")
    .schema(STOCK_PRICE_SCHEMA)
    .load(csv_file_path)
)

In [4]:
df.show(5)

+----------+-------+-------+-------+-------+----+--------+
|      Date|  Price|   Open|   High|    Low|Vol.|Change %|
+----------+-------+-------+-------+-------+----+--------+
|10/01/2024|41.3097|41.2118|41.4322|41.1114|NULL|   0.51%|
|09/01/2024|   41.1|  41.15| 41.583|40.8925|NULL|   0.24%|
|08/01/2024|   41.0|  41.05|41.5253| 40.441|NULL|   0.24%|
|07/01/2024|   40.9|  40.55|41.7096|40.3965|NULL|   1.24%|
|06/01/2024|   40.4|  40.65|40.8663|40.0075|NULL|  -0.25%|
+----------+-------+-------+-------+-------+----+--------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Vol.: string (nullable = true)
 |-- Change %: string (nullable = true)



In [6]:
df = df.drop(STOCK_PRICE_COLUMNS.VOLUME)

In [7]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Change %: string (nullable = true)



In [9]:
df.write.option("header", True).option("delimiter", ",").csv(
    parent_dir + "data/processed/usd_uah_currency.csv"
)